In [1]:
!pip install -r requirements.txt

In [2]:
# from .full.Gensim_Doc_Modelling import *
# import ipynb.fs.defs.Topic_Analysis

from gsdmm_master.gsdmm import MovieGroupProcess
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter


from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from matplotlib import colors as mcolors
import scipy.sparse as sp

In [3]:
DATASET_PATH = 'data/'
AUTHORITY_SET_PATH = 'data/authority_sets.txt'
AUTH_FILTERED_PATH = 'data/authority_filtered/'
LANGUAGE = 'italian'

DATASET = pd.read_csv(DATASET_PATH + 'ferrero_corporate.csv')

C:\Users\alxau\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,4,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
def preprocess_dataset(dataset):
    dataset = dataset.iloc[dataset.date_created_at.dropna().index]
    dataset['date_index'] = pd.to_datetime(dataset.date_created_at, errors='coerce')
    dataset = dataset.loc[dataset['date_index'].dropna().index]
    
    datasetusers = []

    for mdate, df in dataset.groupby(pd.Grouper(key='date_index', freq='d')):
        datasetusers.append(df.loc[df.user_id.drop_duplicates().index])

    return pd.concat(datasetusers)

def Map(func, lst):
    return list(map(func, lst))

def Filter(func, lst):
    return list(filter(func, lst))

def Enumerate(lst):
    return list(enumerate(lst))

In [5]:
def get_data_between(dataset, mindate, maxdate):
    lowerbound = dataset[dataset.date_index > mindate]
    return lowerbound[lowerbound.date_index < maxdate]

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alxau\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
with open(file=AUTHORITY_SET_PATH, mode='r') as f:
    category_list = []
    category = f.readline()[:-1]
    category_list.append(category)
    
    authority_sets = {}
    authority_sets[category] = []
    
    lines = f.readlines()
    for line in lines:
        if(line[0] != '@'):
            if(line != '\n'):
                category = line[:-1]
                category_list.append(category)
                authority_sets[category] = []
        else:
            if(line[-1:] == '\n'):
                authority_sets[category].append(line[1:-1])
            else:
                authority_sets[category].append(line[1:])

for i in authority_sets:
    print(i, authority_sets[i])

categories = Enumerate(category_list)
print(categories)

Politici ['matteosalvinimi', 'GiorgiaMeloni', 'EnricoLetta ', 'nzingaretti ', 'MonicaCirinna ', 'elenabonetti', 'matteorenzi', 'RossellaMuroni', 'luigidimaio', 'ale_dibattista', 'beppe_grillo', 'GiuseppeConteIT', 'NFratoianni', 'pbersani', 'pdnetwork', 'bobogiac', 'gasparripdl ', 'FratellidItalia', 'LegaSalvini', 'Capezzone', 'borghi_claudio', 'berlusconi', 'forza_italia', 'DarioNardella', 'carlaruocco1', 'gualtierieurope', 'BeaLorenzin', 'robersperanza', 'dariofrance', 'DaniloToninelli', 'BeppeSala', 'ellyesse', 'ElioLannutti', 'sbonaccini', 'marcocappato', 'PietroGrasso', 'ItaliaViva', 'zaiapresidente', 'TeresaBellanova', 'Azione_it', 'GuidoCrosetto', 'gparagone']
Esperti di settore e giornalisti ['DarioBressanini', 'sabri_giannini', 'RudyBandiera', 'la_kuzzo', 'M_gabanelli', 'robertosaviano', 'petergomezblog', 'corradoformigli', 'IaconaRiccardo', 'marcotravaglio', 'AndreaScanzi', 'lucatelese', 'stanzaselvaggia', 'concitadeg', 'giucruciani', 'mariogiordano5', 'Tommasolabate', 'DAVIDP

In [8]:
# filters tweets by a subset of authorities
def get_authority_tweets(auth_list, df):
   return df[df['user_username'].isin(auth_list)]

# filters tweets by multiple subsets of authorities and outputs their union
def get_authority_tweets_multiple_categories(auth_cat_list, df):
   dflist = []
   for i, value in enumerate(auth_cat_list):
      authority_set = authority_sets[value]
      dflist.append(df[df['user_username'].isin(authority_set)])
   return pd.concat(dflist).drop_duplicates()

In [9]:
df = preprocess_dataset(DATASET)
df.to_csv(DATASET_PATH + 'ferrero_corp_preprocessed.csv')
df_copy = df.copy()
# ferrero_corp_no_na = ferrero_corpnospam.dropna(1)

<ipython-input-4-ce738b389829>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['date_index'] = pd.to_datetime(dataset.date_created_at, errors='coerce')


In [10]:
get_authority_tweets(['LegaSalvini'], df_copy)

,social,id,topic_id,rule_id,is_comment,text,links,mentions,hashtags,media,...,place_location_lat,place_location_lon,place_lat,place_lon,user_place_country_code,score_spam,score_performance,user_score_rank,score_rating,date_index
32064,twitter,1186949229438263297,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,Siamo in un pericoloso paradosso con il Paese ...,https://t.co/lkkYO9w8VC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-23 10:15:23+00:00
21988,twitter,1230076823947677697,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,Conte sta dimostrando di non essere in grado d...,https://t.co/oZRFaWe8V0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-19 10:29:03+00:00
19039,twitter,1255498355045732353,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,In diretta dal Senato Roberta Ferrero sulla cr...,https://t.co/WFw7lvlViy,NaN,#coronavirus,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-29 14:05:08+00:00
18043,twitter,1263364165672095756,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,Roberta Ferrero (Lega): SITUAZIONE IN ITALIA h...,https://t.co/ebN3CuaJsR,NaN,NaN,https://pbs.twimg.com/media/EYhfA5oXYAAQ_qO.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-21 07:01:03+00:00
17785,twitter,1268207318912638978,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,? INTERVENTO DI ROBERTA FERRERO sulla crisi ec...,https://t.co/8ohZoynFPm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-03 15:46:01+00:00
16277,twitter,1288435615919476736,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,? INTERVENTO DI ROBERTA FERRERO SULLO SCOSTAME...,https://t.co/WKqe3P3H6n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-29 11:26:03+00:00
13686,twitter,1312082692956196867,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"++ ANCHE LA SENATRICE ROBERTA FERRERO, SABATO ...","https://t.co/AgN5yMbQDQ, https://t.co/3gPHsOAy57",NaN,#catania,https://pbs.twimg.com/media/EjV0QafWsAAdrN8.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-02 17:31:05+00:00
13543,twitter,1313152228828631040,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,? PD E M5S PENSANO SOLO A FARE LE LORO MARCHET...,https://t.co/KVubXsSIEN,NaN,#ferrero,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-05 16:21:03+00:00
13074,twitter,1316394843015385089,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,? Il governo Conte deve capire che non si ridu...,https://t.co/xxEaVCB2Q6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-14 15:06:02+00:00
10648,twitter,1338859112075935746,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,? Conte ascolti la #Lega: occorre un taglio dr...,https://t.co/LM9QbEPbqt,NaN,#lega,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-15 14:51:01+00:00


In [11]:
df_copy

,social,id,topic_id,rule_id,is_comment,text,links,mentions,hashtags,media,...,place_location_lat,place_location_lon,place_lat,place_lon,user_place_country_code,score_spam,score_performance,user_score_rank,score_rating,date_index
37450,twitter,1156790362859548672,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"#Miliardari - Giovanni Ferrero, Leonardo Del ...",https://t.co/1Qqt1xE0W2,@berlusconi,#miliardari,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-01 04:54:49+00:00
37449,twitter,1156813511156256775,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,#Arte e #comunicazione aziendale: lo strano r...,https://t.co/Anv1ZB6b2d,@enrica_ferrero,"#arte, #comunicazione",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-01 06:26:48+00:00
37448,twitter,1156825180045492224,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"Cessione #Sampdoria, Gazzetta dello Sport: ""Ba...",https://t.co/vjy88qcA1o,@calcioweb,"#sampdoria, #ferrero, #vialli",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-01 07:13:10+00:00
37447,twitter,1156832449571278849,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,Cessione Sampdoria Gazzetta dello Sport: “Ball...,https://t.co/iuLZt9qNW0,NaN,"#cessione, #sampdoria, #gazzetta, #dello",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-01 07:42:03+00:00
37446,twitter,1156837153432854530,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"Ferrero, 140 milioni per la sua Sampdoria http...","https://t.co/qhYUKnpesY, https://t.co/RDZjCDNvyG",NaN,NaN,https://pbs.twimg.com/media/EA3pQqVWsAAOhuo.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-01 08:00:45+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,twitter,1431211661151637508,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"@eIa_madonna Perché ""ferrero""? ?",NaN,@eia_madonna,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-27 11:07:05+00:00
3,twitter,1431216680806621186,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"@Osvaldo88322280 Un po’ le tengo per me, il re...",NaN,@osvaldo88322280,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-27 11:27:02+00:00
2,twitter,1431222843627409409,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"@gippu1 Preferivi lo stile Commisso, ok? O que...",NaN,@gippu1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-27 11:51:31+00:00
1,twitter,1431237450085584898,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,E dicono di essere pure di sinistra! Ma come r...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-27 12:49:33+00:00


In [12]:
auth_df = get_authority_tweets_multiple_categories(category_list, df)
auth_df

,social,id,topic_id,rule_id,is_comment,text,links,mentions,hashtags,media,...,place_location_lat,place_location_lon,place_lat,place_lon,user_place_country_code,score_spam,score_performance,user_score_rank,score_rating,date_index
34648,twitter,1174618145786650624,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,La Ferrero riconosce a 6000 dipendenti un prem...,https://t.co/T3wBQLWAeo,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-19 09:36:03+00:00
34065,twitter,1176441842096910336,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,Il Ministro Fioramonti si permette di giustifi...,NaN,@lofioramonti,NaN,NaN,...,41.898417,12.545146,41.898417,12.545146,NaN,NaN,NaN,NaN,NaN,2019-09-24 10:22:47+00:00
33847,twitter,1176748033066844160,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,@theoddmother @lofioramonti In Italia la Ferre...,NaN,"@theoddmother, @lofioramonti",NaN,NaN,...,41.898417,12.545146,41.898417,12.545146,NaN,NaN,NaN,NaN,NaN,2019-09-25 06:39:28+00:00
32534,twitter,1183821988437135362,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,@gianni_la_marca @angelofilippi Per il mercato...,NaN,"@gianni_la_marca, @angelofilippi",NaN,NaN,...,41.898417,12.545146,41.898417,12.545146,NaN,NaN,NaN,NaN,NaN,2019-10-14 19:08:51+00:00
32064,twitter,1186949229438263297,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,Siamo in un pericoloso paradosso con il Paese ...,https://t.co/lkkYO9w8VC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-23 10:15:23+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,twitter,1418810115445166080,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"Ferrero, la strategia green del big di Alba (è...","https://t.co/r2XvO05KdN, https://t.co/t8k8vmUlQk",NaN,NaN,https://pbs.twimg.com/media/E7CgSCqXMAAiXZ_.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-24 05:47:46+00:00
841,twitter,1420717382830415880,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,Ferrero l'azienda meglio percepita sul territo...,"https://t.co/4O4c31bPiq, https://t.co/n8RpxZP5Vu",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-29 12:06:34+00:00
822,twitter,1421105542945202182,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"L’attimo fuggente, online il numero di agosto ...",https://t.co/Q9FPElZrOU,"@ferrero_eu, @generali, @prada, @armani, @ciro...","#luigicremonini, #vincenzocremonini, #alberton...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-30 13:48:58+00:00
467,twitter,1426075939201421313,7ee62d5c-5811-45ce-9cef-c09d143f041a,12559.0,False,"Mediobanca, Gavio arrotonda la quota. I Ferrer...","https://t.co/QPX7deAYBt, https://t.co/XUfJMFKsFE",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-13 06:59:33+00:00


In [13]:
auth_df.to_csv(AUTH_FILTERED_PATH + 'ferrero_corp_auth.csv')

In [14]:
# istante in cui gli utenti hanno tweettato, e variazione di volume nel periodo immediatamente adiacente
# 